
## Visual analysis of the 'others'


In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)

import plotly.plotly as py
import plotly.graph_objs as go 
import plotly.colors as pc
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(True)

In [2]:
ps = pd.read_csv('playerscores2017.csv')  # 2017 player box scores
bs = pd.read_csv('boxscores2017.csv')  # 2017 team box scores
len(ps.playDispNm.unique())  # number of unique players

540

In [3]:
# calculate game score
ps['gameScore'] = ps.playPTS + 0.4* ps.playFGM - 0.7 * ps.playFGA - 0.4 * (ps.playFTA - ps.playFTM) + \
                  0.7 * ps.playORB + 0.3 * ps.playDRB + ps.playSTL + 0.7 * ps.playAST + \
                  0.7 * ps.playBLK - 0.4 * ps.playPF - ps.playTO


In [4]:
ptgCnt = ps[['gmDate', 'teamAbbr', 'playDispNm']]\
                    .groupby(['teamAbbr', 'playDispNm'], as_index=False).count()
noTeams = ptgCnt.groupby(['playDispNm'], as_index=False).count()  
traded = noTeams.loc[noTeams.gmDate > 1]  # traded players, ignore for now

gsAvgs = ps[['gameScore', 'teamAbbr', 'playDispNm']]\
                    .groupby(['teamAbbr', 'playDispNm'], as_index=False).mean()
gsAvgs['rank'] = gsAvgs.groupby(['teamAbbr'])['gameScore'].rank(ascending=False)
pLocAvgs = ps[['gameScore', 'teamAbbr', 'playDispNm', 'teamLoc']]\
                    .groupby(['teamAbbr', 'playDispNm', 'teamLoc'], as_index=False).mean()
rankGS = pd.merge(pLocAvgs, gsAvgs, on=['teamAbbr', 'playDispNm'], suffixes=('_loc', '_avg'))

rankGS['locLift'] = (rankGS.gameScore_loc - rankGS.gameScore_avg)*100.0/rankGS.gameScore_avg

maxRank = 14
rankGS = rankGS.loc[(rankGS.gameScore_avg > 0) & (rankGS.gameScore_loc > 0) & (rankGS['rank'] < maxRank)]

rankGS.head()

,teamAbbr,playDispNm,teamLoc,gameScore_loc,gameScore_avg,rank,locLift
4,ATL,Damion Lee,Away,7.922222,7.886667,8.0,0.450831
5,ATL,Damion Lee,Home,7.833333,7.886667,8.0,-0.676247
8,ATL,Dennis Schröder,Away,11.290323,13.305970,1.0,-15.148445
9,ATL,Dennis Schröder,Home,15.041667,13.305970,1.0,13.044494
10,ATL,Dewayne Dedmon,Away,8.060000,8.872581,5.0,-9.158335


In [5]:
teams = ps.teamAbbr.unique()
traces = []
colors = pc.DEFAULT_PLOTLY_COLORS
homeGS = rankGS.loc[rankGS.teamLoc == 'Home'].sort_values('rank')

for i, team in list(enumerate(teams)):
    traces += [go.Scatter(
                x=homeGS.loc[homeGS.teamAbbr == team, 'rank'],
                y=homeGS.loc[homeGS.teamAbbr == team, 'locLift'],
                name=team,
                text=homeGS.loc[homeGS.teamAbbr == team, 'playDispNm'],
                visible=True,
                opacity = 0.15,
                marker=dict(color = colors[i%len(colors)]),
                hoverinfo='none'
                          )
              ] 
    
leagueAvg = homeGS.groupby(['rank'], as_index=False)['locLift'].mean()

traces += [go.Scatter(
            x=leagueAvg['rank'],
            y=leagueAvg['locLift'],
            name='League Avg Lift',
            visible=True,
            opacity = 1,
            marker=dict(color = 'rgb(0,0,0)')
                      )
          ] 

layout = dict(
    title='Home GmSc Lift as a League Average', 
    showlegend=False,
    xaxis=dict(
        title='Average GmSc Rank within Team'
               ),
    yaxis=dict(
        title='% Lift in GmSc Average at Home'
               ),    
              )

fig = dict(data=traces, layout=layout)


iplot(fig, filename='GmSc League')

In [6]:
teams = ps.teamAbbr.unique()
traces = []
colors = pc.DEFAULT_PLOTLY_COLORS
homeGS = rankGS.loc[rankGS.teamLoc == 'Home'].sort_values('rank')

for i, team in list(enumerate(teams)):
    traces += [go.Scatter(
                x=homeGS.loc[homeGS.teamAbbr == team, 'rank'],
                y=homeGS.loc[homeGS.teamAbbr == team, 'locLift'],
                name=team,
                text=homeGS.loc[homeGS.teamAbbr == team, 'playDispNm'],
                visible=True,
                opacity = 1,
                marker=dict(color = colors[i%len(colors)])
                          )
              ] 
    

buttons = []

for i, team in list(enumerate(teams)):
    buttons += [dict(label = team,
                 method = 'update',
                 args = [{'visible': [(n%30) == i for n in range(60)]},
                         {'title': "%s's Home GmSc lift by rank in team" % team},
                         {'opacity': [0.9 for n in range(60)]}
                         ])]
    
buttons += [dict(label = 'Reset',
                 method = 'update',
                 args = [{'visible': [True for n in range(60)]},
                         {'title': "Home GmSc lift by rank in team"},
                         ]
                )]

updatemenus = list([
    dict(active=-1,
         buttons=buttons,
    )
])


layout = dict(
    title='Home GmSc Lift as a league Average', 
    showlegend=False,
    xaxis=dict(
        title='Average GmSc Rank within Team'
               ),
    yaxis=dict(
        title='% Lift in GmSc Average at Home'
               ),    
    updatemenus=updatemenus
              )
fig = dict(data=traces, layout=layout)
iplot(fig, filename='GmSc by team')

In [7]:
bs['team3QScore'] = bs.teamPTS1 + bs.teamPTS2 + bs.teamPTS3
bs['oppt3QScore'] = bs.opptPTS1 + bs.opptPTS2 + bs.opptPTS3

In [8]:
close = 8
closeBS = bs.loc[(abs(bs.team3QScore - bs.oppt3QScore) < close) & (abs(bs.teamPTS - bs.opptPTS) < close)][['gmDate', 'teamAbbr']]

In [9]:
keys = ['gmDate', 'teamAbbr']
i1 = ps.set_index(keys).index
i2 = closeBS.set_index(keys).index
psClose = ps[i1.isin(i2)]

In [10]:
closeAvgs = psClose[['gameScore', 'teamAbbr', 'playDispNm']]\
                    .groupby(['teamAbbr', 'playDispNm'], as_index=False).mean()

closeGS = pd.merge(closeAvgs, gsAvgs, on=['teamAbbr', 'playDispNm'], suffixes=('_close', '_avg'))

closeGS['clutchLift'] = (closeGS.gameScore_close - closeGS.gameScore_avg)*100.0/closeGS.gameScore_avg

closeGS = closeGS.loc[(closeGS.gameScore_avg > 0) & (closeGS.gameScore_close > 0) & (closeGS['rank'] < maxRank)]

In [11]:
teams = ps.teamAbbr.unique()
traces = []
colors = pc.DEFAULT_PLOTLY_COLORS
closeGS = closeGS.sort_values('rank')

for i, team in list(enumerate(teams)):
    traces += [go.Scatter(
                x=closeGS.loc[closeGS.teamAbbr == team, 'rank'],
                y=closeGS.loc[closeGS.teamAbbr == team, 'clutchLift'],
                name=team,
                text=closeGS.loc[closeGS.teamAbbr == team, 'playDispNm'],
                visible=True,
                opacity = 0.15,
                marker=dict(color = colors[i%len(colors)]),
                hoverinfo='none'
                          )
              ] 
    
leagueAvg = closeGS.groupby(['rank'], as_index=False)['clutchLift'].mean()

traces += [go.Scatter(
            x=leagueAvg['rank'],
            y=leagueAvg['clutchLift'],
            name='League Avg Lift',
            visible=True,
            opacity = 1,
            marker=dict(color = 'rgb(0,0,0)')
                      )
          ] 

layout = dict(
    title='Close game GmSc Lift as a League Average', 
    showlegend=False,
    xaxis=dict(
        title='Average GmSc Rank within Team'
               ),
    yaxis=dict(
        title='% Lift in GmSc Average of Close Games',
        range=[-100, 60]
               ),    
              )

fig = dict(data=traces, layout=layout)


iplot(fig, filename='GmSc League')